In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

In [69]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"

In [2]:

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key

In [188]:
end_use = "hs?get=CTY_NAME,CON_VAL_MO,CAL_DUT_MO,I_COMMODITY,I_COMMODITY_SDESC"

surl = "https://api.census.gov/data/timeseries/intltrade/imports/" + end_use 

surl  = surl + my_key + "&time=" + "from+2024-01" + "&COMM_LVL=HS4" 

url = surl + "&CTY_CODE=" + "1220"

# Mexico is 2010
# Canada is 1220
    
r = requests.get(url) 
    
print(r)
    
df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

df.columns = r.json()[0]

df.time = pd.to_datetime(df.time, format="%Y-%m")

df["imports"] = df["CON_VAL_MO"].astype(float)
        
df["hs2"] = df["I_COMMODITY"].astype(str)

df.drop(["CON_VAL_MO", "I_COMMODITY", "COMM_LVL"], axis = 1, inplace = True)

<Response [200]>


In [190]:
df.head()

,CTY_NAME,CAL_DUT_MO,I_COMMODITY_SDESC,time,CTY_CODE,imports,hs2
0,CANADA,0,"HORSES, ASSES, MULES AND HINNIES, LIVE",2024-01-01,1220,3854246.0,0101
1,CANADA,208,"BOVINE ANIMALS, LIVE",2024-01-01,1220,104271503.0,0102
2,CANADA,0,"SWINE, LIVE",2024-01-01,1220,39974869.0,0103
3,CANADA,0,"SHEEP AND GOATS, LIVE",2024-01-01,1220,90724.0,0104
4,CANADA,0,"CHICKENS, DUCKS, GEESE, TURKEYS, AND GUINEAS, ...",2024-01-01,1220,3477554.0,0105


In [191]:
df["share"] = df.imports / df.imports.sum()

In [192]:
grp = df.groupby(["hs2"])

top_products = grp.agg({"imports":"sum", "I_COMMODITY_SDESC":"first", "share": "sum"})

In [193]:
top_products.sort_values(by = ["imports"], ascending = False)[0:20]

,imports,I_COMMODITY_SDESC,share
hs2,,,
2709,8.840692e+10,CRUDE OIL FROM PETROLEUM AND BITUMINOUS MINERALS,0.234427
8703,2.616701e+10,MOTOR CARS & VEHICLES FOR TRANSPORTING PERSONS,0.069387
9801,1.393975e+10,EXPTS OF REPAIRED IMPTS; IMPTS OF RETURNED EXPTS,0.036964
2710,1.164553e+10,OIL (NOT CRUDE) FROM PETROL & BITUM MINERAL ETC.,0.030880
8708,1.057380e+10,PARTS & ACCESS FOR MOTOR VEHICLES (HEAD 8701-8...,0.028038
2711,7.873917e+09,PETROLEUM GASES & OTHER GASEOUS HYDROCARBONS,0.020879
7601,7.090415e+09,"ALUMINUM, UNWROUGHT",0.018802
8704,6.458951e+09,MOTOR VEHICLES FOR TRANSPORT OF GOODS,0.017127
8411,5.514112e+09,"TURBOJETS, TURBOPROPELLERS & OTH GAS TURBINES,...",0.014622


In [216]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["imports"]/1000000000).map('{:,.1f}'.format)
    
    df["hover_label_2"] = (df["share"]*100).map('{:,.1f}'.format)
    
    source = ColumnDataSource(df)
    
    return source

In [232]:
def make_bar_chart(df):

    width = 600
    height = 500

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, title= "Top US Imports from Canada at HS4-level",
           toolbar_location = 'below',
           tools = "reset")
        
    p.vbar(x = "position", top = "imports", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "color",
       source = source)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@name</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Imports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Share of Total: @hover_label_2%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '13pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

    p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = 90000000000
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)
    return p

In [233]:
foo = top_products.sort_values(by = ["imports"], ascending = False)[0:20]

foo["hs-code"] = foo.index

foo["name"] = "HS4 " + foo["hs-code"] + ": " + foo["I_COMMODITY_SDESC"].str[0:30]

foo["color"] = "#ff0000"

In [234]:
p = make_bar_chart(foo)

In [235]:
show(p)

In [239]:
canada = make_bar_chart(foo)
    
output_file('.\\docs\\' + "us_imports_canada.html")

div0 = Div(text = """Each bar represents the total sum of U.S. imports from Canada in 2024 (through November) for an HS4 category. 
The top 20 import categories are displayed. Hover your cursor over each bar to learn more.
""", max_width=600, background = background )
div0.sizing_mode= "scale_both"
        
outfig = column(canada, div0, sizing_mode="scale_both")

show(outfig)